In [1]:
import os
import re
import codecs
import numpy as np
import pandas as pd

In [2]:
train_path = './train/text'
test_path = './test/text'
valid_path = './valid/text'

In [3]:
char2int = {
  u'0' :0, u'1' :1, u'2' :2, u'3' :3, u'4' :4,
  u'5' :5, u'6' :6, u'7' :7, u'8' :8, u'9' :9,
  u'０':0, u'１':1, u'２':2, u'３':3, u'４':4,
  u'５':5, u'６':6, u'７':7, u'８':8, u'９':9,
}

numKanji0 = [ u'', u'一', u'二', u'三', u'四', u'五', u'六', u'七', u'八', u'九' ]
numKanji1 = [ u'', u'',   u'二', u'三', u'四', u'五', u'六', u'七', u'八', u'九' ]
numKanji  = [ numKanji0, numKanji1, numKanji1, numKanji1 ]

numPlace1 = [ u'', u'十', u'百', u'千' ]
numPlace4 = [ u'', u'万', u'億', u'兆', u'京', u'垓' ]

def convert_pure_integerstring(match):
    source = match.group()
    numstr = re.sub( u'[,，]', u'', source )
    #  numKanji[3] = numKanji0    # (b)
    #  numKanji[3] = numKanji1 if len(numstr) == 4 else numKanji0  # (c),(e)
    s = []
    for ch in ((u'0'*((4-len(numstr)%4)&3))+numstr): s.insert(0,char2int[ch])

    list = []
    while len(s):
        temp = u''
        for i in range(4):
            if s[i]: temp = numKanji[i][s[i]] + numPlace1[i] + temp
        list.append(temp)
        s = s[4:]

    if len(list) > len(numPlace4): return source
    result = u''  # (a),(b),(c),(d)
    #  result = list[0][1:] if list[0].startswith(u'一千') else list[0]  # (e)
    #  result = list[0]  # (f)

    for i in range(len(list)):  # (a),(b),(c),(d)
    #  for i in range(1,len(list)):　# (e),(f)
        if list[i]:
            if len(list) > 1 and list[i] == u'千': list[i] = u'一' + list[i]  # (d)
    #        if list[i] == u'千': list[i] = u'一' + list[i]  # (f)
            result = list[i] + numPlace4[i] + result

    return result if result else u'零'

def convert_integerstring(string):
    if string == None or string == u'': return u''
    p = re.compile(u'[0-9０-９][0-9０-９,，]*[0-9０-９]|[0-9０-９]')
    return p.sub( convert_pure_integerstring, string )

In [4]:
string = '人数は10人で、八月十五日に予約してください'
convert_integerstring(string)

'人数は十人で、八月十五日に予約してください'

In [5]:
from tqdm import tqdm
for path in [train_path, test_path, valid_path]:
    with open(path) as f:
        lines = f.readlines()
        
    new_lines = []
    for line in tqdm(lines):
        try:
            idx, text = line.split()
        except:
            print(line)
            print(pre, path)
            print(aaa)
        text_kanji = convert_integerstring(text)
        text_kanji = text_kanji.replace('。', '')
        new_lines.append(idx+' '+text_kanji+'\n')
        pre = line
        
    out_path = path.replace('text', 'text2')
    with codecs.open(out_path,"w","utf-8") as f:
        f.writelines(new_lines)

100%|██████████| 853/853 [00:00<00:00, 116751.77it/s]


In [9]:
new_lines = []
for text in lines:
    t = convert_kanji_to_int(text)
    t = zenkaku2hankaku(t)
    new_lines.append(t)

In [10]:
lines

['U0000_20131101_1_01_001 あのすいませんフランス料理屋を都内で探しているんですけどー。\n',
 'U0000_20131101_1_01_002 えっとーそうだなちょと家族の誕生日用に使うんですけど。\n',
 'U0000_20131101_1_01_003 えーと日暮里の近くで何かいい店ありますかね。\n',
 'U0000_20131101_1_01_004 えっとですね誕生日ー用で家族のそうですね誕生日で。\n',
 'U0000_20131101_1_01_005 えっとですね今ー二十三で。\n',
 'U0000_20131101_1_01_006 で家族五人で行くんーですけど。\n',
 'U0000_20131101_1_01_007 曜日はえと来週の土曜日の夜がいいんですけど。\n',
 'U0000_20131101_1_01_008 六時以降ーならいつでもー空いてる時間大丈夫です。\n',
 'U0000_20131101_1_01_009 一駅なんで大丈夫なんですけどあの駅からなるべくー近いところがはいいいん。\n',
 'U0000_20131101_1_01_010 そうですね同じぐらいの時間なんでえとコースの内容とかーで決めたいんですけど最初のほうのお店っていうのだいたい一コースどのくらいの値段になりますかね。\n',
 'U0000_20131101_1_01_011 ビストロ西菜亭。\n',
 'U0000_20131101_1_01_012 はそうです結構しますねはい。\n',
 'U0000_20131101_1_01_013 もう一個のほうーはどのくらいになりますかね。\n',
 'U0000_20131101_1_01_014 はい。\n',
 'U0000_20131101_1_01_015 五千円から一万円はい。\n',
 'U0000_20131101_1_01_016 できればあの誕生日なんでーあのケーキとかがつく。\n',
 'U0000_20131101_1_01_017 コースとかがいいんですけど。\n',
 'U0000_20131101_1_01_018 そうですね二個目のお店ーのほうーでお願いーできますか。\n',
 'U0000_20131101_1_01_019 デザート込

In [11]:
new_lines

['U0000_20131101_1_01_001 あのすいませんフランス料理屋を都内で探しているんですけどー。\n',
 'U0000_20131101_1_01_002 えっとーそうだなちょと家族の誕生日用に使うんですけど。\n',
 'U0000_20131101_1_01_003 えーと日暮里の近くで何かいい店ありますかね。\n',
 'U0000_20131101_1_01_004 えっとですね誕生日ー用で家族のそうですね誕生日で。\n',
 'U0000_20131101_1_01_005 えっとですね今ー23で。\n',
 'U0000_20131101_1_01_006 で家族5人で行くんーですけど。\n',
 'U0000_20131101_1_01_007 曜日はえと来週の土曜日の夜がいいんですけど。\n',
 'U0000_20131101_1_01_008 6時以降ーならいつでもー空いてる時間大丈夫です。\n',
 'U0000_20131101_1_01_009 1駅なんで大丈夫なんですけどあの駅からなるべくー近いところがはいいいん。\n',
 'U0000_20131101_1_01_010 そうですね同じぐらいの時間なんでえとコースの内容とかーで決めたいんですけど最初のほうのお店っていうのだいたい1コースどのくらいの値段になりますかね。\n',
 'U0000_20131101_1_01_011 ビストロ西菜亭。\n',
 'U0000_20131101_1_01_012 はそうです結構しますねはい。\n',
 'U0000_20131101_1_01_013 もう1個のほうーはどのくらいになりますかね。\n',
 'U0000_20131101_1_01_014 はい。\n',
 'U0000_20131101_1_01_015 5000円から10000円はい。\n',
 'U0000_20131101_1_01_016 できればあの誕生日なんでーあのケーキとかがつく。\n',
 'U0000_20131101_1_01_017 コースとかがいいんですけど。\n',
 'U0000_20131101_1_01_018 そうですね2個目のお店ーのほうーでお願いーできますか。\n',
 'U0000_20131101_1_01_019 デ

In [ ]:
with codecs.open(path_text,"a","utf-8") as f:
    f.write(text_list[j])